In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('blast.db')
c = conn.cursor()

In [32]:
def get_table_name(word_size):
    return "preprocess_wordsize_" + str(word_size)
def get_index_table_name(word_size):
    return "indexed_words_wordsize_" + str(word_size)

In [3]:
def all_combinations(word_size):
    nucleotides = ('A','C','G','T')
    sol = list()
    
    def all_combinations_rec(size,cur_sol):
    
        # exit condition
        if size == 0:
            sol.append(''.join(cur_sol))
            return
        
        for nuc in nucleotides:
            
            cur_sol.append(nuc)
            all_combinations_rec(size-1,cur_sol)
            
            # backtrack
                       
            cur_sol.pop()
    
    all_combinations_rec(word_size,[])
    
    return sol

In [90]:
def create_word_table(word_size):
    
    # new table name
    new_table_name = get_table_name(word_size)
    
    # Create new table for this word size
#     c.execute("DROP TABLE {}".format(new_table_name))
    
    create_table_s = "CREATE TABLE {} (id INTEGER PRIMARY KEY,word varchar({}) NOT NULL UNIQUE)".format(new_table_name,str(word_size))
    c.execute(create_table_s)
    
    # insert all possible words into table
    all_combinations_nucleotides = all_combinations(word_size)
    
    for word in all_combinations_nucleotides:
        # Insert a row of data
        insert_word_s = "INSERT INTO {} (word) VALUES (?)".format(new_table_name)
        c.execute(insert_word_s,(word,))
        
    
    # create the related table containing the index in the sequence of a certain word
    
    new_index_table_name = get_index_table_name(word_size)
    index_table_id_name = new_index_table_name + '_id'
#     c.execute("DROP TABLE {}".format(new_index_table_name))
    create_index_table_s = "CREATE TABLE {} (id INTEGER PRIMARY KEY,sequence_index int NOT NULL,word_index int NOT NULL,FOREIGN KEY (word_index) REFERENCES {}(id))".format(new_index_table_name,new_table_name)
    c.execute(create_index_table_s)
    
    
    # Save (commit) the changes
    conn.commit()

In [92]:
# makes a cursor which is like an iterator
c.execute("SELECT * FROM preprocess_wordsize_5")

In [93]:
# you can do this to print them or make them into a list
for row in c.execute("SELECT * FROM preprocess_wordsize_5"):
    print(row)

(1, 'AAAAA')
(2, 'AAAAC')
(3, 'AAAAG')
(4, 'AAAAT')
(5, 'AAACA')
(6, 'AAACC')
(7, 'AAACG')
(8, 'AAACT')
(9, 'AAAGA')
(10, 'AAAGC')
(11, 'AAAGG')
(12, 'AAAGT')
(13, 'AAATA')
(14, 'AAATC')
(15, 'AAATG')
(16, 'AAATT')
(17, 'AACAA')
(18, 'AACAC')
(19, 'AACAG')
(20, 'AACAT')
(21, 'AACCA')
(22, 'AACCC')
(23, 'AACCG')
(24, 'AACCT')
(25, 'AACGA')
(26, 'AACGC')
(27, 'AACGG')
(28, 'AACGT')
(29, 'AACTA')
(30, 'AACTC')
(31, 'AACTG')
(32, 'AACTT')
(33, 'AAGAA')
(34, 'AAGAC')
(35, 'AAGAG')
(36, 'AAGAT')
(37, 'AAGCA')
(38, 'AAGCC')
(39, 'AAGCG')
(40, 'AAGCT')
(41, 'AAGGA')
(42, 'AAGGC')
(43, 'AAGGG')
(44, 'AAGGT')
(45, 'AAGTA')
(46, 'AAGTC')
(47, 'AAGTG')
(48, 'AAGTT')
(49, 'AATAA')
(50, 'AATAC')
(51, 'AATAG')
(52, 'AATAT')
(53, 'AATCA')
(54, 'AATCC')
(55, 'AATCG')
(56, 'AATCT')
(57, 'AATGA')
(58, 'AATGC')
(59, 'AATGG')
(60, 'AATGT')
(61, 'AATTA')
(62, 'AATTC')
(63, 'AATTG')
(64, 'AATTT')
(65, 'ACAAA')
(66, 'ACAAC')
(67, 'ACAAG')
(68, 'ACAAT')
(69, 'ACACA')
(70, 'ACACC')
(71, 'ACACG')
(72, 'ACACT')
(

In [18]:
l = list(c.execute("SELECT * FROM preprocess_wordsize_5"))

In [31]:
c.execute("SELECT * FROM indexed_words_wordsize_5")

In [37]:
def get_indexes_for_word(word):
    word_size = (len(word))
    new_table_name = get_table_name(word_size)
    new_index_table_name = get_index_table_name(word_size)
    
    s = "SELECT * FROM {} LEFT JOIN {} where {}.id = {}.word_index AND word = ?".format(new_index_table_name,new_table_name,new_table_name,new_index_table_name)
    c.execute(s,(word,))

In [94]:
get_indexes_for_word('AAAAA')

In [ ]:
def insert_index_for_word(word,index):
    word_size = (len(word))
    new_table_name = get_table_name(word_size)
    new_index_table_name = get_index_table_name(word_size)
    
    s = "INSERT INTO {} VALUES "